# German credit scoring [sklearn]

By running this notebook, you’ll create a whole test suite in a few lines of code. The model used here is a simple logistic regression model with the german credit scoring dataset. Feel free to use your own model (tabular, text, or LLM).

You’ll learn how to:
* Detect vulnerabilities by scanning the model
* Generate a test suite with domain-specific tests
* Customize your test suite by loading a test from the Giskard catalog
* Upload your model to the Giskard server to:
    * Compare models to decide which one to promote
    * Debug your tests to diagnose issues
    * Share your results and collect business feedback from your team

## Install Giskard

In [94]:
!pip install giskard>=2.0.0-b1

## Import libraries

In [95]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import giskard
from giskard import Model, Dataset, testing, GiskardClient

## Define constants

In [96]:
# Constants.
COLUMN_TYPES = {
    "account_check_status": "category",
    "duration_in_month": "numeric",
    "credit_history": "category",
    "purpose": "category",
    "credit_amount": "numeric",
    "savings": "category",
    "present_employment_since": "category",
    "installment_as_income_perc": "numeric",
    "sex": "category",
    "personal_status": "category",
    "other_debtors": "category",
    "present_residence_since": "numeric",
    "property": "category",
    "age": "category",
    "other_installment_plans": "category",
    "housing": "category",
    "credits_this_bank": "numeric",
    "job": "category",
    "people_under_maintenance": "numeric",
    "telephone": "category",
    "foreign_worker": "category",
}

TARGET_COLUMN_NAME = "default"

# Paths.
DATA_URL = "https://raw.githubusercontent.com/Giskard-AI/giskard-examples/main/datasets/credit_scoring_classification_model_dataset/german_credit_prepared.csv"

## Dataset preparation

### Load data

In [97]:
df = pd.read_csv(DATA_URL, keep_default_na=False, na_values=["_GSK_NA_"])

### Train-test split

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=TARGET_COLUMN_NAME), df[TARGET_COLUMN_NAME],
                                                    test_size=0.2, random_state=0, stratify=df[TARGET_COLUMN_NAME])

### Wrap dataset with Giskard

In [99]:
raw_data = pd.concat([X_test, Y_test], axis=1)
wrapped_data = Dataset(raw_data,
                       name='German credit scoring dataset',
                       target=TARGET_COLUMN_NAME,
                       column_types=COLUMN_TYPES)

/Users/mykytaalekseiev/Work/giskard_ai_test_v2_merged/python-client/giskard/core/dataset_validation.py:122: UserWarning: Feature 'people_under_maintenance' is declared as 'numeric' but has 2 (<= category_threshold=2) distinct values. Are you sure it is not a 'category' feature?
  warning(


## Model training

### Define preprocessing steps

In [100]:
columns_to_scale = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "numeric"]
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

columns_to_encode = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "category"]
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False)),
])

preprocessor = ColumnTransformer(transformers=[
        ("num", numeric_transformer, columns_to_scale),
        ("cat", categorical_transformer, columns_to_encode),
])

### Build estimator

In [101]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=100))
])

pipeline.fit(X_train, Y_train)

pred_train = pipeline.predict(X_train)
pred_test = pipeline.predict(X_test)

print(classification_report(Y_test, pred_test))

              precision    recall  f1-score   support

     Default       0.68      0.45      0.54        60
 Not default       0.79      0.91      0.85       140

    accuracy                           0.77       200
   macro avg       0.73      0.68      0.69       200
weighted avg       0.76      0.77      0.75       200



### Wrap model with Giskard

In [102]:
wrapped_model = Model(model=pipeline,
                      model_type="classification",
                      name="Credit scoring classifier",
                      feature_names=list(COLUMN_TYPES.keys()),
                      classification_threshold=0.5,
                      classification_labels=pipeline.classes_.tolist())

# Validate wrapped model.
print(classification_report(Y_test, pipeline.classes_[wrapped_model.predict(wrapped_data).raw_prediction]))

              precision    recall  f1-score   support

     Default       0.68      0.45      0.54        60
 Not default       0.79      0.91      0.85       140

    accuracy                           0.77       200
   macro avg       0.73      0.68      0.69       200
weighted avg       0.76      0.77      0.75       200



## Scan your model to find vulnerabilities
With the Giskard scan feature, you can detect vulnerabilities in your model, including performance biases, unrobustness, data leakage, stochasticity, underconfidence, ethical issues, and more. For detailed information about the scan feature, please refer to our scan documentation.

In [104]:
results = giskard.scan(wrapped_model, wrapped_data)

Your model is successfully validated.
🔎 Running scan…
2023-06-06 12:59:26,139 pid:12217 MainThread giskard.scanner.logger INFO     Running detectors: ['PerformanceBiasDetector', 'TextPerturbationDetector', 'EthicalBiasDetector', 'DataLeakageDetector', 'StochasticityDetector', 'OverconfidenceDetector', 'UnderconfidenceDetector']
Running detector PerformanceBiasDetector…2023-06-06 12:59:26,141 pid:12217 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Running
2023-06-06 12:59:26,141 pid:12217 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Calculating loss
2023-06-06 12:59:26,366 pid:12217 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Loss calculated (took 0:00:00.223666)
2023-06-06 12:59:26,367 pid:12217 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: Finding data slices
2023-06-06 12:59:28,461 pid:12217 MainThread giskard.scanner.logger INFO     PerformanceBiasDetector: 36 slices found (took 0:00:02.092861

In [105]:
display(results)

<iframe id="scan-5272785472" srcdoc="
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:collapse;border-color:inherit;text-indent:0}button,input,optgroup,select,textarea{color:inherit;font-family:inherit;font-size:100%

In [109]:
test_suite = results.generate_test_suite("My first test suite")
test_suite.run()

Executed 'Precision on data slice “`other_installment_plans` == "bank"”' with arguments {'model': <giskard.models.sklearn.SKLearnModel object at 0x13ac29270>, 'dataset': <giskard.datasets.base.Dataset object at 0x13a8ac850>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x13a9c1690>, 'threshold': 0.7540625}: 
               Test failed
               Metric: 0.6
               
               
Executed 'Precision on data slice “`account_check_status` == "0 <= ... < 200 DM"”' with arguments {'model': <giskard.models.sklearn.SKLearnModel object at 0x13ac29270>, 'dataset': <giskard.datasets.base.Dataset object at 0x13a8ac850>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x13a9fd450>, 'threshold': 0.7540625}: 
               Test failed
               Metric: 0.6
               
               
Executed 'Precision on data slice “`present_employment_since` == "... < 1 year"”' with arguments {'model': <giskard.models.sklearn.SKL

(False,
 [('Precision on data slice “`other_installment_plans` == "bank"”', 
                  Test failed
                  Metric: 0.6
                  
                  ),
  ('Precision on data slice “`account_check_status` == "0 <= ... < 200 DM"”',
   
                  Test failed
                  Metric: 0.6
                  
                  ),
  ('Precision on data slice “`present_employment_since` == "... < 1 year"”',
   
                  Test failed
                  Metric: 0.65
                  
                  ),
  ('Recall on data slice “`personal_status` == "divorced"”',
   
                  Test failed
                  Metric: 0.8
                  
                  ),
  ('Precision on data slice “`property` == "if not A121/A122 : car or other, not in attribute 6"”',
   
                  Test failed
                  Metric: 0.72
                  
                  ),
  ('Precision on data slice “`sex` == "female"”',
   
                  Test failed
                  Metric: 0.74
                  
                  ),
  ('Precision on data slice “`savings` == "... < 100 DM"”',
   
                  Test failed
                  Metric: 0.74
                  
                  )])

## Customize your suite by loading objects from the Giskard catalog

The Giskard open source catalog will enable to load:
* Tests such as metamorphic, performance, prediction & data drift, statistical tests, etc
* Slicing functions such as detectors of toxicity, hate, emotion, etc
* Transformation functions such as generators of typos, paraphrase, style tune, etc

For demo purposes, we will load a simple unit test (test_f1) that checks if the test F1 score is above the given threshold. For more examples of tests and functions, refer to the Giskard catalog.

In [110]:
test_suite.add_test(testing.test_f1(model=wrapped_model, dataset=wrapped_data, threshold=0.7)).run()

Executed 'Precision on data slice “`other_installment_plans` == "bank"”' with arguments {'model': <giskard.models.sklearn.SKLearnModel object at 0x13ac29270>, 'dataset': <giskard.datasets.base.Dataset object at 0x13a8ac850>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x13a9c1690>, 'threshold': 0.7540625}: 
               Test failed
               Metric: 0.6
               
               
Executed 'Precision on data slice “`account_check_status` == "0 <= ... < 200 DM"”' with arguments {'model': <giskard.models.sklearn.SKLearnModel object at 0x13ac29270>, 'dataset': <giskard.datasets.base.Dataset object at 0x13a8ac850>, 'slicing_function': <giskard.slicing.slice.QueryBasedSliceFunction object at 0x13a9fd450>, 'threshold': 0.7540625}: 
               Test failed
               Metric: 0.6
               
               
Executed 'Precision on data slice “`present_employment_since` == "... < 1 year"”' with arguments {'model': <giskard.models.sklearn.SKL

(False,
 [('Precision on data slice “`other_installment_plans` == "bank"”', 
                  Test failed
                  Metric: 0.6
                  
                  ),
  ('Precision on data slice “`account_check_status` == "0 <= ... < 200 DM"”',
   
                  Test failed
                  Metric: 0.6
                  
                  ),
  ('Precision on data slice “`present_employment_since` == "... < 1 year"”',
   
                  Test failed
                  Metric: 0.65
                  
                  ),
  ('Recall on data slice “`personal_status` == "divorced"”',
   
                  Test failed
                  Metric: 0.8
                  
                  ),
  ('Precision on data slice “`property` == "if not A121/A122 : car or other, not in attribute 6"”',
   
                  Test failed
                  Metric: 0.72
                  
                  ),
  ('Precision on data slice “`sex` == "female"”',
   
                  Test failed
                  Metric: 0.74
                  
                  ),
  ('Precision on data slice “`savings` == "... < 100 DM"”',
   
                  Test failed
                  Metric: 0.74
                  
                  ),
  ('F1',
   
                  Test succeed
                  Metric: 0.85
                  
                  )])

## Upload your suite to the Giskard server

Upload your suite to the Giskard server to:
* Compare models to decide which model to promote
* Debug your tests to diagnose the issues
* Create more domain-specific tests that are integrating business feedback
* Share your results

In [ ]:
# Uploading the test suite will automatically save the model, dataset, tests, slicing & transformation functions inside the Giskard UI server
# Create a Giskard client after having install the Giskard server (see documentation)
token = "API_TOKEN"  # Find it in Settings in the Giskard server

client = GiskardClient(
    url="http://localhost:19000",  # URL of your Giskard instance
    token=token
)

my_project = client.create_project("my_project", "PROJECT_NAME", "DESCRIPTION")

# Upload to the current project ✉️
test_suite.upload(client, "my_project")